In [51]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

import time
from tqdm import tqdm


np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_data = 100
num_features = 5
num_items = 4

cuda


In [52]:


def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    x = rnd.normal(0, 1, (n, m, p))
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    
    w = rnd.normal(0, 1, (m, p))
    b = rnd.normal(0, 1, (n, m))
    r = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            r[i, j] = np.dot(w[j], x[i, j]) + b[i, j]

    r = 1 / (1 + np.exp(-r))

    return x, r, c, Q

In [53]:
class optModel:
    """
    This is a class for optimization models.
    """

    def __init__(self, r, c, Q, alpha):
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data = num_data
        self.num_items = num_items
        
    def __repr__(self):
        return 'optModel ' + self.__class__.__name__
        

    def setObj(self, r, c):

        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)

    
    def solve(self, closed=False):
        """
        A method to solve the optimization problem for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization
            closed (bool): solving the problem in closed form

        Returns:
            tuple: optimal solution and optimal value
        """
        if closed:
            return self.solveC()

        self.d = cp.Variable(self.num_items)
        self.setObj(self.r, self.c)
        self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
        opt_sol = self.d.value
        opt_val = self.problem.value

        return opt_sol, opt_val


    def solveC(self):
        """
        A method to solve the optimization problem in closed form for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization

        Returns:
            tuple: optimal solution and optimal value
        """
        r = self.r
        c = self.c
        if self.alpha == 1:
            raise ValueError("Work in progress")
        
        S = np.sum(c ** (1 - 1 / self.alpha) * r ** (-1 + 1 / self.alpha))
        opt_sol_c = (c ** (-1 / self.alpha) * r ** (-1 + 1 / self.alpha) * self.Q) / S
        opt_val_c = np.sum((r * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)

        return opt_sol_c, opt_val_c

In [54]:
class optDataset(Dataset):
    """
    This class is Torch Dataset class for optimization problems.
    """

    def __init__(self, features, costs, r, Q, alpha=0.5, closed=False):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): optimization model
            features (np.ndarray): features
            c (np.ndarray): c of objective function
            r (np.ndarray): r of objective function
            Q (float): budget
            alpha (float): alpha of objective function
            closed (bool): solving the problem in closed form

        """
        self.feats = features
        self.costs = costs
        self.r = r
        self.Q = Q
        self.alpha = alpha
        self.closed = closed

        self.sols, self.objs = self._getSols()

    def _getSols(self):
        """
        A method to get the solutions of the optimization problem
        """
        opt_sols = []
        opt_objs = []
        
        for i in tqdm(range(len(self.costs))):
            sol, obj = self._solve(self.r[i], self.costs[i])
            opt_sols.append(sol)
            opt_objs.append([obj])
        
        return np.array(opt_sols), np.array(opt_objs)

    def  _solve(self, r, c):
        """
        A method to solve the optimization problem to get oan optimal solution with given r and c

        Args:
            r (np.ndarray): r of objective function
            c (np.ndarray): c of objective function

        Returns:
            tuple: optimal solution (np.ndarray), objective value (float)
        """
        self.model = optModel(r, c, self.Q, self.alpha)
        if self.closed:
            return self.model.solveC()
        else:
            return self.model.solve()

    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)
    
    def __getitem__(self, index):
        """
        A method to retrieve data

        Args:
            index (int): data index

        Returns:
            tuple: data features (torch.tensor), costs (torch.tensor), optimal solutions (torch.tensor) and objective values (torch.tensor)
        """
        return (
            torch.FloatTensor(self.feats[index]), # x 
            torch.FloatTensor(self.costs[index]), # c
            torch.FloatTensor(self.r[index]), # r 
            torch.FloatTensor(self.sols[index]),# optimal solution
            torch.FloatTensor(self.objs[index]), # objective value
        )

In [55]:
x, r, c, Q = genData(num_data, num_features, num_items)
optmodel = optModel(r, c, Q, alpha=0.5)
# sols = []
# objs = []
# print("Solving the optimization problem...")
# time.sleep(1)

# for i in tqdm(range(num_data)):
#     sol, obj = optmodel.solve(r[i], c[i])

#     sols.append(sol)
#     objs.append([obj])

In [56]:
from sklearn.model_selection import train_test_split

x_train, x_test, c_train, c_test, r_train, r_test = train_test_split(x, c, r, test_size=0.2, random_state=42)

In [57]:
dataset_train = optDataset(x_train, c_train, r_train, Q, alpha=0.5, closed=False)
dataset_test = optDataset(x_test, c_test, r_test, Q, alpha=0.5, closed=False)

100%|██████████| 20/20 [00:00<00:00, 293.41it/s]


In [58]:
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [59]:
class LinearRegressionOne(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionOne, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out
    
class LinearRegressionModel(nn.Module):
    def __init__(self, num_items, num_features):
        super(LinearRegressionModel, self).__init__()
        self.num_items = num_items
        self.num_features = num_features
        self.linears = nn.ModuleList([nn.Linear(num_features, 1) for _ in range(num_items)])

    def forward(self, x):
        outputs = []
        for i in range(self.num_items):
            outputs.append(torch.sigmoid(self.linears[i](x[:, i, :])))
        return torch.cat(outputs, dim=1)

model = LinearRegressionModel(num_items, num_features)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

LinearRegressionModel(
  (linears): ModuleList(
    (0-3): 4 x Linear(in_features=5, out_features=1, bias=True)
  )
)

In [60]:
# load data
for data in loader_train:
    x, c, r, d, z  = data
    if next(model.parameters()).is_cuda:
        x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
        
    break
print(z[0].item())

10.900128364562988


In [61]:
def regret(predmodel, optmodel, dataloader):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # evaluate
    predmodel.eval()
    loss = 0
    optsum = 0
    # load data
    for data in dataloader:
        x, c, r, d, z  = data
        # cuda
        if next(predmodel.parameters()).is_cuda:
            x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
        # predict
        with torch.no_grad(): # no grad
            rp = predmodel(x).to("cpu").detach().numpy()
        # solve
        for j in range(rp.shape[0]):
            # accumulate loss
            loss += calRegret(optModel, c[j].to("cpu").detach().numpy(), rp[j], r[j].to("cpu").detach().numpy(),
                              z[j].item())
        optsum += abs(z).sum().item()
    # turn back train mode
    predmodel.train()
    # normalized
    return loss / (optsum + 1e-7)

def objValue(d, r, alpha):
    """
    A function to calculate objective value
    """
    if alpha == 1:
        return np.sum(np.log(np.multiply(r, d)))
    else:
        return np.sum(np.power(np.multiply(r, d), 1 - alpha)) / (1 - alpha)



def calRegret(optmodel, cost, pred_r, true_r, true_obj):
    """
    A function to calculate normalized true regret for a batch

    Args:
        optmodel (optModel): optimization model
        pred_cost (torch.tensor): predicted costs
        true_cost (torch.tensor): true costs
        true_obj (torch.tensor): true optimal objective values

    Returns:predmodel
        float: true regret losses
    """
    # opt sol for pred cost
    model = optmodel(pred_r, cost, Q, alpha=0.5)
    sol, _ = model.solve()
    # obj with true cost
    obj = objValue(sol, true_r, alpha=0.5)
    # loss
    loss = true_obj - obj
    return loss

In [62]:
temp = regret(model, optmodel, loader_test)
temp

0.04270446719710229

In [63]:
from matplotlib import pyplot as plt

def visLearningCurve(loss_log, loss_log_regret):
    # create figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))

    # draw plot for training loss
    ax1.plot(loss_log, color="c", lw=2)
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iters", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve on Training Set", fontsize=16)

    # draw plot for regret on test
    ax2.plot(loss_log_regret, color="royalblue", ls="--", alpha=0.7, lw=2)
    ax2.set_xticks(range(0, len(loss_log_regret), 2))
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_ylim(0, 0.5)
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret", fontsize=16)
    ax2.set_title("Learning Curve on Test Set", fontsize=16)

    plt.show()

In [64]:
from torch.autograd import Function
import numpy as np
from pathos.multiprocessing import ProcessingPool

In [65]:
from torch.autograd import Function

class RegretLossFunction(Function):
    @staticmethod
    def forward(ctx, pred_r, true_r, cost, true_obj, optmodel):
        device = pred_r.device

        # Detach inputs to avoid modifying the original tensors
        pred_r_np = pred_r.detach().cpu().numpy()
        true_r_np = true_r.detach().cpu().numpy()
        cost_np = cost.detach().cpu().numpy()
        true_obj_np = true_obj.detach().cpu().numpy()

        # Calculate the optimal solution for predicted costs
        opt_solutions = []
        opt_values = []
        for i in range(pred_r_np.shape[0]):
            optmodel = optmodel(pred_r_np[i], cost_np[i], Q, alpha=0.5)
            sol, val = optmodel.solve()
            opt_solutions.append(sol)
            opt_values.append(val)

        opt_solutions = np.array(opt_solutions)
        opt_values = np.array(opt_values)

        # Calculate the objective value with true costs
        obj_values = []
        for i in range(pred_r_np.shape[0]):
            obj_val = objValue(opt_solutions[i], true_r_np[i], alpha=0.5)
            obj_values.append(obj_val)

        obj_values = np.array(obj_values)

        # Calculate the regret loss
        loss = true_obj_np - obj_values

        # Save for backward pass
        ctx.save_for_backward(pred_r, torch.FloatTensor(opt_solutions).to(device))
        ctx.optmodel = optmodel

        return torch.FloatTensor(loss).to(device)

    @staticmethod
    def backward(ctx, grad_output):
        pred_r, opt_solutions = ctx.saved_tensors

        # Compute gradients using autograd
        pred_r.requires_grad_(True)
        with torch.enable_grad():
            optmodel = ctx.optmodel
            cost = pred_r.new_tensor(ctx.cost, requires_grad=True)
            true_r = pred_r.new_tensor(ctx.true_r, requires_grad=True)
            true_obj = pred_r.new_tensor(ctx.true_obj, requires_grad=True)
            loss = RegretLossFunction.apply(pred_r, true_r, cost, true_obj, optmodel)

        loss.backward()
        grad_pred_r = pred_r.grad

        return grad_pred_r, None, None, None, None


In [76]:
def regretLoss(optmodel, pred_r, true_r, cost, true_obj):
    return RegretLossFunction.apply(pred_r, true_r, cost, true_obj, optmodel)


In [77]:
# loss = 0
# for epoch in range(num_epochs):
 
#     # start time
#     tick = time.time()
#     # load data
#     for i, data in enumerate(loader_train):
#         x, c, r, d, z  = data
#         if torch.cuda.is_available():
#             x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
#         rp = predmodel(x)


            

In [78]:

# # train mode
# predmodel.train()
# # init log
# loss_log = []
# loss_log_regret = [regret(predmodel, optmodel, loader_test)]
# # init elapsed time
# elapsed = 0

# for epoch in range(num_epochs):
#     # start time
#     tick = time.time()
#     # load data
#     for i, data in enumerate(loader_train):
#         x, c, r, d, z  = data
#         # cuda
#         if torch.cuda.is_available():
#             x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
#         # forward pass
#         loss = torch.nn.MSELoss()
#         # backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # record time
#         tock = time.time()
#         elapsed += tock - tick
#         # log
#         loss_log.append(loss.item())
#     regret_loss = regret(predmodel, optmodel, loader_test)
#     loss_log_regret.append(regret_loss)
#     print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, loss.item(), regret*100))
# print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))

In [85]:
def trainModel(predmodel, optmodel, loss_func, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(predmodel.parameters(), lr=lr)
    # train mode
    predmodel.train()
    # init log
    loss_log = []
    loss_log_regret = [regret(predmodel, optmodel, loader_test)]
    # init elapsed time
    elapsed = 0

    for epoch in range(num_epochs):
        # start time
        tick = time.time()
        # load data
        for i, data in enumerate(loader_train):
            x, c, r, d, z  = data
            # cuda
            if torch.cuda.is_available():
                x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
            # forward pass
            rp = predmodel(x)
            for j in range(rp.shape[0]):
                loss += loss_func(optmodel, rp[j], r[j], c[j], z[j])
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            # log
            loss_log.append(loss.item())
        regret_loss = regret(predmodel, optmodel, loader_test)
        loss_log_regret.append(regret_loss)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, loss.item(), regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))

    return loss_log, loss_log_regret

In [86]:
# Set up the model, optimizer, and MSE loss
model = LinearRegressionModel(num_items, num_features).to(device)
optmodel = optModel(r, c, Q, alpha=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.05)
criterion = nn.MSELoss()

# Run the training
loss_log, loss_log_regret = trainModel(model, optmodel, criterion, num_epochs=20, lr=1e-2)

# Visualize the learning curves
visLearningCurve(loss_log, loss_log_regret)


TypeError: trainModel() got multiple values for argument 'num_epochs'